In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：自定义训练表格回归模型以进行批量预测

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_tabular_regression_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_tabular_regression_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_tabular_regression_batch.ipynb">
      在 Google Cloud 笔记本中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex SDK来训练和部署用于批量预测的自定义表格回归模型。

### 数据集

本教程使用的数据集是[Boston房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。您在本教程中使用的数据集版本已内置于TensorFlow中。训练好的模型预测房屋的中位价格，以1K美元为单位。

### 目标

在本教程中，您将使用Vertex SDK在Google预构建的Docker容器中，从Python脚本中创建一个带有训练管道的自定义模型，然后对上传的模型进行批量预测。您也可以使用`gcloud`命令行工具或在Cloud Console上在线创建自定义模型。

执行的步骤包括：

- 创建用于训练模型的Vertex自定义作业。
- 训练TensorFlow模型。
- 检索并加载模型工件。
- 查看模型评估。
- 将模型上传为Vertex`Model`资源。
- 进行批量预测。

成本

本教程使用谷歌云的可计费组件：

* Vertex AI
* 云存储

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[云存储价格](https://cloud.google.com/storage/pricing)，并使用[Pricing 计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

### 配置本地开发环境

如果您正在使用Colab或Google Cloud笔记本，则您的环境已满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境符合此笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

[设置Python开发环境的Cloud Storage指南](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了简化的指导：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端窗口中运行`pip3 install jupyter`。

5. 要启动Jupyter，请在终端窗口中运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

安装

安装最新版本的Python版Vertex SDK。

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

### 重新启动内核

安装了额外的软件包之后，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU 运行时

本教程不需要 GPU 运行时。

### 设置您的 Google Cloud 项目

**无论您使用的是什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300 的免费信用额度，可用于支付计算/存储成本。

2. [确保您的项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI API、Compute Engine API 和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，则需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 运行以 `!` 为前缀的行作为shell命令，并插入以 `$` 为前缀的 Python 变量。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于本笔记本其余部分的操作。以下是Vertex AI支持的区域。我们建议您选择离您最近的区域。

- 美洲: `us-central1`
- 欧洲: `europe-west4`
- 亚太地区: `asia-east1`

您可能无法使用多区域存储桶进行Vertex AI训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行在线教程会话，您可能会使用共享的测试帐户或项目。为了避免用户之间在创建的资源上出现名称冲突，您可以为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的 Google Cloud 账户

**如果您正在使用 Google Cloud 笔记本**，则您的环境已经经过身份验证。请跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证以通过oAuth进行身份验证。

**否则**，请按照以下步骤操作：

在 Cloud Console 中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

**单击创建服务帐户**。

在**服务帐户名称**字段中输入名称，然后单击**创建**。

在**将此服务帐户授予项目访问权限**部分，单击角色下拉列表。 在过滤框中键入“Vertex”，然后选择 **Vertex 管理员**。在过滤框中键入“Storage Object Admin”，然后选择 **Storage Object Admin**。

单击创建。 包含密钥的 JSON 文件将下载到您的本地环境。

在下面的单元格中，将您的服务帐户密钥的路径作为 GOOGLE_APPLICATION_CREDENTIALS 变量输入，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

创建一个云存储桶

**无论您的笔记本环境如何，都需要执行以下步骤。**

当您初始化用于 Python 的 Vertex SDK 时，您需要指定一个云存储临时存储桶。临时存储桶是您的数据集和模型资源相关数据在会话之间保留的地方。

请在下方设置您的云存储桶的名称。存储桶名称必须在所有谷歌云项目中是全局唯一的，包括您的组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的桶尚不存在时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查存储桶的内容来验证对云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

为您的项目和相应的存储桶初始化Python的Vertex SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量 `TRAIN_GPU/TRAIN_NGPU` 和 `DEPLOY_GPU/DEPLOY_NGPU` 设置为使用支持 GPU 的容器镜像以及分配给虚拟机实例的 GPU 数量。例如，要使用具有 4 个 Nvidia Telsa K80 GPU 的容器镜像分配给每个虚拟机实例，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定 `(None, None)` 来使用一个在 CPU 上运行的容器镜像。

在这里了解有关您地区的硬件加速器支持的更多信息。

*注意*: TF 2.3 之前的版本对于 GPU 支持将无法在本教程中加载自定义模型。这是一个已知问题，在 TF 2.3 中已经修复 -- 这是由于在服务功能中生成的静态图操作所导致的。如果在您自己的自定义模型中遇到此问题，请使用带有 GPU 支持的 TF 2.3 容器镜像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预先构建的容器

设置用于训练和预测的预先构建的 Docker 容器镜像。


获取最新列表，请查看[用于训练的预先构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。


获取最新列表，请查看[用于预测的预先构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量 `TRAIN_COMPUTE` 和 `DEPLOY_COMPUTE` 来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`: 每个vCPU 3.75GB 内存。
     - `n1-highmem`: 每个vCPU 6.5GB 内存
     - `n1-highcpu`: 每个vCPU 0.9GB 内存
 - `vCPUs`: \[2, 4, 8, 16, 32, 64, 96 \] 数量

*注意: 以下不支持用于训练的选项:*

 - `standard`: 2 vCPUs
 - `highcpu`: 2, 4 和 8 vCPUs

*注意: 您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在您已经准备好开始创建自己的定制模型并对波士顿房屋进行训练。

### 检查培训包

#### 包布局

在开始培训之前，您将了解如何为自定义培训任务组装Python包。解压后，包含以下目录/文件布局的包。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将包安装到Docker镜像的操作环境中的说明。

文件 `trainer/task.py` 是用于执行自定义培训任务的Python脚本。*注意*，当我们在工作池规格中引用时，我们将目录斜杠替换为点(`trainer.task`)并去掉文件后缀(`.py`)。

#### 包组装

在以下单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Boston Housing tabular regression\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 内容

在下一个单元格中，您将编写训练脚本 task.py 的内容。我不会详细说明，只是让您浏览一下。总结如下：

- 从命令行（`--model_dir`）获取要保存模型文件的目录，如果未指定，则从环境变量 `AIP_MODEL_DIR` 获取。
- 从 TF.Keras 内置数据集加载波士顿房屋数据集。
- 使用 TF.Keras 模型 API 构建一个简单的深度神经网络模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分布策略。
- 用由 `args.epochs` 指定的 epochs 训练模型（`fit()`）。
- 将训练好的模型保存到指定的模型目录（`save(args.model_dir)`）。
- 将每个特征的最大值保存到指定的参数文件中 `f.write(str(params))`。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for Boston Housing

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))


def make_dataset():

  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)

  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)


# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr))
  return model

NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
BATCH_SIZE = 16
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_dnn_model()

# Train the model
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=GLOBAL_BATCH_SIZE)
model.save(args.model_dir)

将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成一个压缩的tar包，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_boston.tar.gz

### 创建并运行自定义训练任务

要训练一个自定义模型，您需要执行两个步骤：1）创建一个自定义训练任务，2）运行该任务。

#### 创建自定义训练任务

通过`CustomTrainingJob`类创建一个自定义训练任务，包括以下参数：

- `display_name`：自定义训练任务的可读名称。
- `container_uri`：训练容器镜像。
- `requirements`：训练容器镜像的包要求（例如，pandas）。
- `script_path`：训练脚本的相对路径。

In [ ]:
job = aip.CustomTrainingJob(
    display_name="boston_" + TIMESTAMP,
    script_path="custom/trainer/task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["gcsfs==0.7.1", "tensorflow-datasets==4.4"],
)

print(job)

### 准备您的命令行参数

现在定义您的自定义训练容器的命令行参数：

- `args`：要传递给作为容器入口点的可执行文件的命令行参数。
  - `--model-dir`：对于我们的演示，我们使用这个命令行参数来指定存储模型工件的位置。
      - 直接：将云存储位置作为命令行参数传递给您的训练脚本（设置变量`DIRECT = True`），或者
      - 间接：服务将云存储位置作为环境变量`AIP_MODEL_DIR`传递给您的训练脚本（设置变量`DIRECT = False`）。在这种情况下，您告诉服务作业规范中的模型工件位置。
  - `"--epochs=" + EPOCHS`：训练的时代数。
  - `"--steps=" + STEPS`：每个时代的步数。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_NAME, TIMESTAMP)

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]

开始运行自定义训练任务

接下来，通过调用方法`run`并传入以下参数来运行自定义任务以启动训练任务：

- `args`：传递给训练脚本的命令行参数。
- `replica_count`：用于训练的计算实例数量（replica_count = 1表示单节点训练）。
- `machine_type`：计算实例的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作器副本的加速器数量。
- `base_output_dir`：用于写入模型工件的云存储位置。
- `sync`：是否阻塞直到作业完成。

In [ ]:
if TRAIN_GPU:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=True,
    )
else:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=True,
    )

model_path_to_deploy = MODEL_DIR

加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在云存储桶中。现在从云存储桶中加载它，然后您可以执行一些操作，如评估模型和进行预测。

要加载模型，您可以使用 TF.Keras 的 `model.load_model()` 方法，将保存模型的云存储路径传递给它 -- 由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

local_model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在让我们找出模型有多好。

### 载入评估数据

您将利用`tf.keras.datasets`中的`load_data()`方法加载波士顿房屋测试（留置）数据。此方法返回一个由两个元素组成的元组，第一个元素是训练数据，第二个是测试数据。每个元素也是一个由两个元素组成的元组：特征数据和对应的标签（自住房屋的中位价值）。

您不需要训练数据，这就是为什么我们将其载入为`(_,_)`。

在对数据进行评估之前，您需要对其进行预处理：

`x_test`：
1. 将每列中的数据标准化（重新缩放），方法是将每个值除以该列的最大值。这将用32位浮点数在0和1之间取代每个单个值。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import boston_housing

(_, _), (x_test, y_test) = boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)


def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float32)
    return feature


# Let's save one data item that has not been scaled
x_test_notscaled = x_test[0:1].copy()

for _ in range(13):
    x_test[_] = scale(x_test[_])
x_test = x_test.astype(np.float32)

print(x_test.shape, x_test.dtype, y_test.shape)
print("scaled", x_test[0])
print("unscaled", x_test_notscaled)

### 进行模型评估

现在评估一下定制工作中的模型表现如何。

In [ ]:
local_model.evaluate(x_test, y_test)

获取服务函数签名

您可以通过重新加载模型并查询每个层对应的签名来获取模型输入和输出层的签名。

在进行预测请求时，您需要将请求路由到服务函数而不是模型，因此您需要知道服务函数的输入层名称--这在您进行预测请求时会用到。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

## 上传模型

接下来，使用`Model.upload()`方法将您的模型上传到`Model`资源，需要提供以下参数：

- `display_name`: `Model`资源的可读名称。
- `artifact`: 训练模型文件的云存储位置。
- `serving_container_image_uri`: 用于提供服务的容器镜像。
- `sync`: 是否异步或同步执行上传操作。

如果以异步方式运行`upload()`方法，您可以随后使用`wait()`方法阻塞直至完成。

In [ ]:
model = aip.Model.upload(
    display_name="boston_" + TIMESTAMP,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=False,
)

model.wait()

获取测试项目

您将使用数据集的测试（保留）部分的示例作为测试项目。

In [ ]:
test_item_1 = x_test[0]
test_label_1 = y_test[0]
test_item_2 = x_test[1]
test_label_2 = y_test[1]
print(test_item_1.shape)

### 创建批处理输入文件

现在创建一个批处理输入文件，您将把它存储在您本地的云存储桶中。预测请求中的每个实例都是以下形式的字典条目：

                        {serving_input: content}

- `serving_input`：底层模型的输入层的名称。
- `content`：测试项目的特征值列表。

In [ ]:
import json

gcs_input_uri = BUCKET_NAME + "/" + "test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {serving_input: test_item_1.tolist()}
    f.write(json.dumps(data) + "\n")
    data = {serving_input: test_item_2.tolist()}
    f.write(json.dumps(data) + "\n")

### 发起批量预测请求

现在您的模型资源已经训练好了，您可以通过调用 batch_predict() 方法来进行批量预测，使用以下参数：

- `job_display_name`: 批量预测作业的可读名称。
- `gcs_source`: 一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`: 用于存储批量预测结果的云存储位置。
- `instances_format`: 输入实例的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`: 输出预测的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `machine_type`: 用于训练的机器类型。
- `accelerator_type`: 硬件加速器类型。
- `accelerator_count`: 附加到工作机复制品的加速器数量。
- `sync`: 如果设置为True，则调用将阻塞，等待异步批处理作业完成。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

batch_predict_job = model.batch_predict(
    job_display_name="boston_" + TIMESTAMP,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_NAME,
    instances_format="jsonl",
    predictions_format="jsonl",
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=DEPLOY_GPU,
    accelerator_count=DEPLOY_NGPU,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    sync=False,
)

print(batch_predict_job)

### 等待批量预测作业完成

接下来，等待批量作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批量预测作业完成。

In [ ]:
batch_predict_job.wait()

获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将被写入您在批处理预测请求中指定的云存储输出桶中。您可以调用方法iter_outputs()获取包含结果的每个云存储文件的列表。每个文件都以JSON格式包含一个或多个预测请求：

- `instance`：预测请求。
- `prediction`：预测响应。

In [ ]:
import json

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- AutoML 训练作业
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME